# Agentic fighters

## Exercise 2 - Creating our own agent

**Un agente es un sistema que percibe su entorno, toma decisiones y emprende acciones de forma autónoma.**

![LLM vs agent](./../images/LLM_vs_agent.png)

### 2.1 - ReAct

The ReAct agent model is a framework designed to integrate the reasoning capabilities of large language models (LLMs) with the ability to take actionable steps, creating a more sophisticated system that can understand and process information, evaluate situations, take appropriate actions, communicate responses, and track ongoing situations.

**ReAct: Reason + Action**

In [1]:
import os

In [2]:
if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY environment variable")

LLM_MODEL = "gpt-4o-mini"
LLM_TEMPERATURE = 0

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_openai import ChatOpenAI

In [4]:
base_model = ChatOpenAI(model=LLM_MODEL, temperature=LLM_TEMPERATURE)

In [5]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append(SystemMessage(system))

    def __call__(self, message):
        self.messages.append(HumanMessage(message))
        result = self.execute()
        self.messages.append(AIMessage(result))

        return result

    def execute(self):
        completion = base_model.invoke(self.messages)

        return completion.content

In [6]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [7]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier":
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [8]:
abot = Agent(prompt)
abot

In [9]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I need to find the average weight of a Toy Poodle using the average_dog_weight action. 
Action: average_dog_weight: Toy Poodle
PAUSE


In [10]:
result = average_dog_weight("Toy Poodle")
print(result)

a toy poodles average weight is 7 lbs


In [11]:
next_prompt = f"Observation: {result}"
next_prompt

'Observation: a toy poodles average weight is 7 lbs'

In [12]:
abot(next_prompt)

'Answer: A Toy Poodle weighs 7 lbs.'

In [13]:
abot.messages

[SystemMessage(content='You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs', additional_kwargs={}, response_metadata={}),
 HumanMessag

In [14]:
abot = Agent(prompt)

In [15]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

"Thought: I need to find the average weights of both a Border Collie and a Scottish Terrier to calculate their combined weight. I'll start by looking up the weight of a Border Collie.  \nAction: average_dog_weight: Border Collie  \nPAUSE"

In [16]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [17]:
abot(next_prompt)

'Thought: Now that I have the weight of the Border Collie, I need to find the average weight of a Scottish Terrier to calculate the combined weight of both dogs.  \nAction: average_dog_weight: Scottish Terrier  \nPAUSE'

In [18]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [19]:
abot(next_prompt)

'Thought: I now have the average weights of both dogs: the Border Collie weighs 37 lbs and the Scottish Terrier weighs 20 lbs. I can calculate their combined weight by adding these two values together.  \nAction: calculate: 37 + 20  \nPAUSE'

In [20]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [21]:
abot(next_prompt)

'Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.'

### 2.2 - Añadimos un loop

Para realmente generar un agente, debemos añadirle un loop para que decida él mismo que acciones tomar y cómo interpretar las cosas.

In [22]:
import re

action_re = re.compile('^Action: (\w+): (.*)$')   #expresión regular para seleccionar acciones

In [23]:
def query(question, max_turns=5):
    bot = Agent(prompt)
    next_prompt = question

    i = 0
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)

        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(f" -- running {action} {action_input}")

            observation = known_actions[action](action_input)
            print("Observation:", observation)

            next_prompt = f"Observation: {observation}"
        else:
            return

In [24]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I need to find the average weights of both a Border Collie and a Scottish Terrier to calculate their combined weight. I'll use the average_dog_weight action for both breeds. 
Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: Now I need to find the average weight of a Scottish Terrier to complete the calculation for their combined weight. 
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: I now have the average weights for both dogs: a Border Collie weighs 37 lbs and a Scottish Terrier weighs 20 lbs. I can calculate their combined weight by adding these two values together. 
Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.
